In [55]:
import pandas as pd
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

data = pd.read_csv('songs.csv', delimiter=',', encoding='latin1')
data

,Name,Artist,Album,Popularity,Lyrics
0,Imagine - Remastered 2010,John Lennon,Imagine,79,8 ContributorsDiscographie 2021 â Pop & Rock...
1,A Whiter Shade Of Pale,Procol Harum,A Whiter Shade Of Pale,0,38 ContributorsA Whiter Shade of Pale Lyrics[I...
2,My Sweet Lord,George Harrison,All Things Must Pass (Remastered),0,58 ContributorsMy Sweet Lord Lyrics[Chorus]\nM...
3,God Only Knows - Mono,The Beach Boys,Pet Sounds (Original Mono & Stereo Mix),67,1 ContributorGod Only Knows (mono mix) LyricsI...
4,Bridge Over Troubled Water,Simon & Garfunkel,Bridge Over Troubled Water,72,57 ContributorsBridge Over Troubled Water Lyri...
...,...,...,...,...,...
794,Thinking out Loud,Ed Sheeran,x (Deluxe Edition),85,192 ContributorsTranslationsTÃ¼rkÃ§eEspaÃ±olPo...
795,Sorry,Justin Bieber,Purpose (Deluxe),83,194 ContributorsTranslationsEspaÃ±olPortuguÃªs...
796,Lean On (feat. MÃ & DJ Snake),Major Lazer,Major Lazer Essentials,30,3 ContributorsTop Tracks of 2016 - Global Lyri...
797,Take Me To Church,Hozier,The Singer Songwriter,25,339 ContributorsTranslationsÙØ§Ø±Ø³ÛEspaÃ±ol...


In [56]:
data = data[data['Lyrics'] != 'Lyrics not found']
data = data[data['Lyrics'].str.len() >= 500]
data = data.drop(0)
data = data.reset_index(drop=True)


In [57]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Configure l'authentification avec Spotify
client_credentials_manager = SpotifyClientCredentials(client_id='1e10e9e3ab3844a8acf959b9fdaa86df', client_secret='babcb04988054942a225ebf446f0ba6f')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Recherche de la piste de référence
track = sp.search(q='walygator', type='track')

if track['tracks']['items']:
    reference_track_id = track['tracks']['items'][0]['id']

    # Obtenir des recommandations basées sur la piste de référence
    recommendations = sp.recommendations(seed_tracks=[reference_track_id])

    for track in recommendations['tracks']:
        print(track['name'], "-", track['artists'][0]['name'])
else:
    print("Track not found.")


3095 pt2 - Alpha Wann
Magma - ISHA
Holla Back - Heskis
Au max - H JeuneCrack
Sud-Est - Infinit'
OSEF - Keroué
La familia - ISHA
PAS D'EFFORT - okis
CBD - Saboteur
SOLEIL PLUVIEUX - Yvnnis
Sonatine - Gen
Sous vide - Di-Meh
80mg - Jeune Mort
Step #5 - Étoiles - Stony Stone
FIDJI - Malo
Emploi fictif - 1863
Avertisseurs (Part II) - Prince Waly
Justesse - Sheldon
COQUILLAGE - Deen Burbigo
IAN WRIGHT - Yvnnis


In [58]:
import pandas as pd
from transformers import pipeline

def get_majority_emotion(lyrics_data):
    # Divisez le texte en lignes de paroles
    lyrics_lines = lyrics_data.split('\n')
    
    # Initialisez le modèle de sentiment
    sent_pipeline = pipeline("sentiment-analysis")

    # Créez une liste pour stocker les sentiments de chaque ligne
    emotions = []

    # Pour chaque ligne de paroles, exécutez le modèle de sentiment et obtenez le sentiment majoritaire
    for line in lyrics_lines:
        sentiment = sent_pipeline(line)
        majority_emotion = max(sentiment, key=lambda x: x['score'])
        emotions.append(majority_emotion['label'])

    # Trouvez l'émotion majoritaire parmi toutes les émotions
    majority_emotion = max(emotions, key=emotions.count)

    return majority_emotion

# Exemple d'utilisation avec un DataFrame
datatest = data.head(10)

datatest['Emotion'] = datatest['Lyrics'].apply(get_majority_emotion)

# Le DataFrame datatest maintenant a une colonne "Emotion" contenant l'émotion majoritaire pour chaque entrée dans la colonne "Lyrics"
datatest

ModuleNotFoundError: No module named 'transformers.utils'

In [59]:
dftest = data.head(1)
dftest

,Name,Artist,Album,Popularity,Lyrics
0,A Whiter Shade Of Pale,Procol Harum,A Whiter Shade Of Pale,0,38 ContributorsA Whiter Shade of Pale Lyrics[I...


In [60]:
name = dftest['Name'].values[0]
name

'A Whiter Shade Of Pale'

In [61]:
def get_audio_features(data, name):
    # Recherche la chanson sur Spotify
    results = sp.search(q=name, type='track', limit=1)
    if results and 'tracks' in results and 'items' in results['tracks'] and len(results['tracks']['items']) > 0:
        track = results['tracks']['items'][0]
        
        # Récupère les URI Spotify pour la chanson
        uri = track['uri']

        # Récupère les données audio de la chanson
        audio_features = sp.audio_features(uri)[0]
        
        # Récupère les données de la piste
        artist = track['album']['artists'][0]['name']
        
        # Crée un nouveau DataFrame avec les données de la chanson
        new_data = pd.DataFrame({
            'artist': [artist],
            'track': [track['name']],
            'danceability': [audio_features['danceability']],
            'energy': [audio_features['energy']],
            'key': [audio_features['key']],
            'loudness': [audio_features['loudness']],
            'mode': [audio_features['mode']],
            'speechiness': [audio_features['speechiness']],
            'acousticness': [audio_features['acousticness']],
            'instrumentalness': [audio_features['instrumentalness']],
            'liveness': [audio_features['liveness']],
            'valence': [audio_features['valence']],
            'tempo': [audio_features['tempo']],
            'duration_ms': [audio_features['duration_ms']]
        })

        # Effectuez une jointure (merge) entre les DataFrames en utilisant une colonne d'identifiant
        data = pd.merge(data, new_data, left_index=True, right_index=True, suffixes=('', '_new'))
        data = data.drop([col for col in data.columns if col.endswith('_new')], axis=1)

        return data
    else:
        return None

df_testo = data.head(2)



In [62]:
def add(datas):
    result_df = datas.copy()  # Créez une copie du DataFrame d'origine

    for index, row in datas.iterrows():
        name = row['Name']  # Assurez-vous que la colonne contenant les noms des chansons est correcte

        # Recherche la chanson sur Spotify
        results = sp.search(q=name, type='track', limit=1)
        if results and 'tracks' in results and 'items' in results['tracks'] and len(results['tracks']['items']) > 0:
            track = results['tracks']['items'][0]

            # Récupère les URI Spotify pour la chanson
            uri = track['uri']

            # Récupère les données audio de la chanson
            audio_features = sp.audio_features(uri)[0]

            # Récupère les données de la piste
            artist = track['album']['artists'][0]['name']

            # Crée un dictionnaire avec les nouvelles données
            new_data = {
                'danceability': audio_features['danceability'],
                'energy': audio_features['energy'],
                'key': audio_features['key'],
                'loudness': audio_features['loudness'],
                'mode': audio_features['mode'],
                'speechiness': audio_features['speechiness'],
                'acousticness': audio_features['acousticness'],
                'instrumentalness': audio_features['instrumentalness'],
                'liveness': audio_features['liveness'],
                'valence': audio_features['valence'],
                'tempo': audio_features['tempo'],
                'duration_ms': audio_features['duration_ms']
            }

            # Ajoute les nouvelles données au DataFrame de résultats
            result_df.loc[index, 'danceability'] = new_data['danceability']
            result_df.loc[index, 'energy'] = new_data['energy']
            result_df.loc[index, 'key'] = new_data['key']
            result_df.loc[index, 'loudness'] = new_data['loudness']
            result_df.loc[index, 'mode'] = new_data['mode']
            result_df.loc[index, 'speechiness'] = new_data['speechiness']
            result_df.loc[index, 'acousticness'] = new_data['acousticness']
            result_df.loc[index, 'instrumentalness'] = new_data['instrumentalness']
            result_df.loc[index, 'liveness'] = new_data['liveness']
            result_df.loc[index, 'valence'] = new_data['valence']
            result_df.loc[index, 'tempo'] = new_data['tempo']
            result_df.loc[index, 'duration_ms'] = new_data['duration_ms']

    return result_df

In [63]:
df_testo = data.head(2)

res = add(df_testo)

res

,Name,Artist,Album,Popularity,Lyrics,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,A Whiter Shade Of Pale,Procol Harum,A Whiter Shade Of Pale,0,38 ContributorsA Whiter Shade of Pale Lyrics[I...,0.249,0.66,0.0,-6.904,1.0,0.0343,0.514,0.00271,0.0891,0.427,149.822,248947.0
1,My Sweet Lord,George Harrison,All Things Must Pass (Remastered),0,58 ContributorsMy Sweet Lord Lyrics[Chorus]\nM...,0.539,0.69,1.0,-9.019,0.0,0.0332,0.083,0.00000,0.0847,0.553,120.983,281232.0


In [64]:
dataframe_with_more_keys = add(data)

In [65]:
dataframe_with_more_keys.shape

(774, 17)

In [66]:
def fetch_song_features(song_name, artist_name):
    # Search for the song using name and artist
    results = sp.search(q=f'track:{song_name} artist:{artist_name}', type='track', limit=1)

    if results['tracks']['items']:
        track_info = results['tracks']['items'][0]
        track_id = track_info['id']

        # Fetch audio features for the track
        audio_features = sp.audio_features(track_id)
        
        if audio_features:
            return audio_features[0]

    return None

# User input for song name and artist
song_name = input("Enter the song name: ")
artist_name = input("Enter the artist name: ")

# Fetch song features
song_features = fetch_song_features(song_name, artist_name)

if song_features:
    # Print the fetched features
    print(f"Song Features for '{song_name}' by {artist_name}:")
    print(f"Danceability: {song_features['danceability']}")
    print(f"Energy: {song_features['energy']}")
    print(f"Key: {song_features['key']}")
    print(f"Loudness: {song_features['loudness']}")
    print(f"Mode: {song_features['mode']}")
    print(f"Speechiness: {song_features['speechiness']}")
    print(f"Acousticness: {song_features['acousticness']}")
    print(f"Instrumentalness: {song_features['instrumentalness']}")
    print(f"Liveness: {song_features['liveness']}")
    print(f"Valence: {song_features['valence']}")
    print(f"Tempo: {song_features['tempo']}")
    print(f"Duration (ms): {song_features['duration_ms']}")
else:
    print("Song not found or features unavailable.")

Song Features for 'betoile' by oboy:
Danceability: 0.603
Energy: 0.268
Key: 10
Loudness: -12.761
Mode: 1
Speechiness: 0.0559
Acousticness: 0.636
Instrumentalness: 0
Liveness: 0.154
Valence: 0.538
Tempo: 172.033
Duration (ms): 210253


In [126]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors

# Assuming dataframe_with_more_keys contains your dataset
# Drop the non-numeric columns
def nearest_neighbours(df, song_name, artist_name):
    numeric_data = df.drop(['Name', 'Artist', 'Album', 'Popularity', 'Lyrics', 'genre'], axis=1)

    # User-entered song features (obtained using fetch_song_features)
    user_song_features = fetch_song_features(song_name, artist_name)

    # Create a DataFrame for the user's song with only numeric features
    user_song_df = pd.DataFrame([user_song_features], columns=numeric_data.columns)

    # Calculate the nearest neighbors
    k = 5  # Number of nearest neighbors to find
    nbrs = NearestNeighbors(n_neighbors=k, algorithm='ball_tree').fit(numeric_data)
    distances, indices = nbrs.kneighbors(user_song_df)

    # Get the 5 nearest songs based on indices
    nearest_songs = df.iloc[indices[0]]
    print("5 Nearest Songs to the User's Song:")
    print(nearest_songs)







In [113]:
def get_artist_genre(artist_name):
    # Use the Spotify API to get the genre of the artist
    artist = sp.search(q=artist_name, type='artist')
    if 'artists' in artist and 'items' in artist['artists'] and len(artist['artists']['items']) > 0:
        genres = artist['artists']['items'][0]['genres']
        if genres:
            return genres
    return None


res = get_artist_genre("drake")

print(res)


['canadian hip hop', 'canadian pop', 'hip hop', 'pop rap', 'rap']


In [69]:
def add_genre_column(dataset):
    # Create an empty list to store the genre for each song
    genres = []

    # Iterate through the rows of the dataset and fetch the genre for each song
    for index, row in dataset.iterrows():
        artist_name = row['Artist']
        genre = get_artist_genre(artist_name)
        genres.append(genre)

    # Add the list of genres as a new column in the dataset
    dataset['genre'] = genres

In [70]:
add_genre_column(dataframe_with_more_keys)

In [71]:
dataframe_with_more_keys.head()

,Name,Artist,Album,Popularity,Lyrics,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,genre
0,A Whiter Shade Of Pale,Procol Harum,A Whiter Shade Of Pale,0,38 ContributorsA Whiter Shade of Pale Lyrics[I...,0.249,0.660,0.0,-6.904,1.0,0.0343,0.514,0.002710,0.0891,0.427,149.822,248947.0,"[album rock, art rock, blues rock, british blu..."
1,My Sweet Lord,George Harrison,All Things Must Pass (Remastered),0,58 ContributorsMy Sweet Lord Lyrics[Chorus]\nM...,0.539,0.690,1.0,-9.019,0.0,0.0332,0.083,0.000000,0.0847,0.553,120.983,281232.0,"[album rock, beatlesque, classic rock, folk ro..."
2,God Only Knows - Mono,The Beach Boys,Pet Sounds (Original Mono & Stereo Mix),67,1 ContributorGod Only Knows (mono mix) LyricsI...,0.521,0.487,6.0,-7.525,0.0,0.0299,0.253,0.000000,0.0579,0.483,117.072,173040.0,"[baroque pop, classic rock, folk rock, mellow ..."
3,Bridge Over Troubled Water,Simon & Garfunkel,Bridge Over Troubled Water,72,57 ContributorsBridge Over Troubled Water Lyri...,0.149,0.206,3.0,-13.888,1.0,0.0323,0.822,0.000649,0.1150,0.264,79.764,293120.0,"[classic rock, folk, folk rock, melancholia, m..."
4,Bohemian Rhapsody - Remastered 2011,Queen,A Night At The Opera (2011 Remaster),0,1 ContributorTop Groups of 2021 LyricsBTS - Bu...,0.392,0.402,0.0,-9.961,0.0,0.0536,0.288,0.000000,0.2430,0.228,143.883,354320.0,"[classic rock, glam rock, rock]"


In [128]:
import pandas as pd

def find_songs_with_genres(dataframe, genres_to_find):
    matching_songs = []

    for index, row in dataframe.iterrows():
        song_genres = row['genre']
        if song_genres is not None and any(genre in song_genres for genre in genres_to_find):
            matching_songs.append(row)

    if matching_songs:
        matching_songs_df = pd.DataFrame(matching_songs)
        return matching_songs_df
    else:
        return pd.DataFrame()

# Example usage:
genres_to_find = ['detroit hip hop', 'hip hop', 'rap'] # Replace with the list of genres you want to search for
matching_songs_df = find_songs_with_genres(dataframe_with_more_keys, genres_to_find)

if not matching_songs_df.empty:
    print("Songs with matching genres:")
    print(matching_songs_df)
else:
    print("No songs found with the specified genres.")


Songs with matching genres:
                                          Name                Artist   
73          California Love - Original Version                  2Pac  \
84               All Eyez On Me (ft. Big Syke)                  2Pac   
86                              Tha Crossroads  Bone Thugs-N-Harmony   
92          How Do U Want It (ft. K-Ci & JoJo)                  2Pac   
98   Only God Can Judge Me (ft. Rappin' 4-Tay)                  2Pac   
..                                         ...                   ...   
684                          Super Freaky Girl           Nicki Minaj   
695         See You Again (feat. Charlie Puth)           Wiz Khalifa   
712                          679 (feat. Monty)             Fetty Wap   
744                              Hotline Bling                 Drake   
747                                        Ayo           Chris Brown   

                                         Album  Popularity   
73                               Greatest Hit

In [136]:
def recommend_song_genre_neighbour(song_name, artist_name):
    genre = get_artist_genre(artist_name)
   

    genre_df = find_songs_with_genres(dataframe_with_more_keys,genre)
    print()

    

    nearest_neighbours(genre_df,song_name, artist_name)


recommend_song_genre_neighbour("paint in black", "the rolling stones")


    

Max Retries reached


SpotifyException: http status: 429, code:-1 - /v1/audio-features/?ids=1RJeiAIwR9pZBgJA8ndZLL:
 Max Retries, reason: too many 429 error responses

In [133]:
csv_filename = "dataframe_with_more_keys.csv"

# Export the DataFrame to a CSV file
dataframe_with_more_keys.to_csv(csv_filename, index=False)